<a href="https://colab.research.google.com/github/Olid07/CSE499A/blob/main/Paper_summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 13.1 MB/s eta 0:00:00


In [ ]:
import datasets

## Save into CSV

In [ ]:
# Convert to Pandas DataFrame
df = pd.DataFrame(dataset['train'])

# Save as CSV
df.to_csv("imdb_dataset.csv", index=False)


NameError: ignored

In [ ]:
# Convert to Pandas DataFrame
df = pd.DataFrame(dataset['validation'])

# Save as CSV
df.to_csv("sci_valid_dataset.csv", index=False)


In [ ]:
# Convert to Pandas DataFrame
df = pd.DataFrame(dataset['test'])

# Save as CSV
df.to_csv("sci_test_dataset.csv", index=False)

In [ ]:
import os
print("Current working directory:", os.getcwd())

In [ ]:
!unzip "/content/drive/MyDrive/Dataset/sci_papers.zip" -d "/content/dataset/"

In [ ]:
! sudo cp -v -r "/content/drive/MyDrive/dataset/sci_papers.zip" "/content/dataset"

In [ ]:
from datasets import load_dataset

dataset = load_dataset('scientific_papers', 'pubmed')

## Dataset dowload from huggingface

In [ ]:
from datasets import get_dataset_split_names

get_dataset_split_names("scientific_papers", 'pubmed')

In [ ]:
train_dataset = dataset['train'] ['abstract']

## Connect to drive

In [ ]:
%cd ..
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.makedirs("/content/dataset")

## Start

In [ ]:
!pip install transformers

In [ ]:
!pip install pytorch_lightning

In [ ]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [ ]:
import json
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from pathlib import  Path
from datasets import load_dataset, load_metric


import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from transformers import pipeline , set_seed, AutoModelForSeq2SeqLM, AutoTokenizer

from tqdm.auto import tqdm
import nltk
from nltk.tokenize import sent_tokenize

nltk.download('punkt')

In [ ]:
! sudo cp -v -r "/content/drive/MyDrive/Dataset/sci_papers.zip" "/content/dataset"

In [ ]:
!unzip "/content/dataset/sci_papers.zip" -d "/content/dataset/"

In [ ]:
df= pd.read_csv("/content/dataset/sci_papers/sci_valid_dataset.csv")


In [ ]:
df_train['article'][0]

## Exploratory Data Analysis(EDA)

In [ ]:
df.info()

In [ ]:
df_train.isnull().sum()

In [ ]:
df.head()

In [ ]:
df_train.describe()

In [ ]:
#abstract word count per row
df_train['abstract_word_count'] = df_train['abstract'].apply(lambda x: len(x.strip().split()))
# number of unique words in body
df_train['article_unique_words']=df_train['article'].apply(lambda x:len(set(str(x).split())))
# word count in body
df_train['article_word_count'] = df_train['article'].apply(lambda x: len(str(x).strip().split()))

df_train.head()

In [ ]:
df_train['article_word_count'].max()

In [ ]:
# Check if NULL values are present in the dataset along with count of the NULL values
for col in df.columns:
    print(col, df[col].isnull().sum())

In [ ]:
df = df.dropna()

In [ ]:
df.info()

In [ ]:
df.drop(columns = 'section_names',  inplace=True)

In [ ]:
# Creation of Dataset and Dataloader
# Defining the train size. So 80% of the data will be used for training and the rest will be used for validation.
train_size = 0.8
df_train=df.sample(frac=train_size,random_state = 42)
df_test=df.drop(df_train.index).reset_index(drop=True)
df_train =df_train.reset_index(drop=True)

In [ ]:
df_train.shape

In [ ]:
device  = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
model = 'Dagar/t5-small-science-papers'
tokenizer = AutoTokenizer.from_pretrained(model)

In [ ]:
model_bart = AutoModelForSeq2SeqLM.from_pretrained(model).to(device)

In [ ]:

def generate_batch_sized_chunks(list_of_elements, batch_size):

  """split the dataset into smaller batches that we can process simultaneously
  Yield successive batch-sized chunks from list_of_elements."""
  for i in range(0, len(list_of_elements), batch_size):

    yield list_of_elements[i : i + batch_size]


In [ ]:
def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=4, device=device,
                               column_text="article",
                               column_summary="abstract"):
  article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
  target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

  for article_batch, target_batch in tqdm(zip(article_batches, target_batches), total=len(article_batches)):


    inputs = tokenizer(article_batch, max_length=1024,  truncation=True,padding="max_length", return_tensors="pt")
    summaries = model.generate(input_ids=inputs["input_ids"].to(device),attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=256)

    ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

        # Finally, we decode the generated texts,
        # replace the  token, and add the decoded texts with the references to the metric.
    decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,clean_up_tokenization_spaces=True)
               for s in summaries]

    decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


    metric.add_batch(predictions=decoded_summaries, references=target_batch)

  #  Finally compute and return the ROUGE scores.
  score = metric.compute()
  return score


In [ ]:
article_token_len = len([tokenizer.encode(s) for s in df_train['article']])

summary_token_len = len([tokenizer.encode(s) for s in df_test['abstract']])


fig, axes = plt.subplots(1, 2, figsize=(10, 4))
axes[0].hist(article_token_len, bins = 20, color = 'C0', edgecolor = 'C0' )
axes[0].set_title("Article Token Length")
axes[0].set_xlabel("Length")
axes[0].set_ylabel("Count")

axes[1].hist(summary_token_len, bins = 20, color = 'C0', edgecolor = 'C0' )
axes[1].set_title("Summary Token Length")
axes[1].set_xlabel("Length")
plt.tight_layout()
plt.show()

In [ ]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['article'] , max_length = 1024, truncation = True )

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['summary'], max_length = 128, truncation = True )

    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }
dataset_pubmed_pt = df.applymap(convert_examples_to_features)



In [ ]:
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_bart)

In [ ]:
%cd /content/drive/MyDrive/Dataset/NLP

In [ ]:
!pip install -U accelerate
!pip install -U transformers

In [ ]:
device

In [ ]:

from transformers import TrainingArguments, Trainer
trainer_args = TrainingArguments(
    output_dir='bart-pubmed', num_train_epochs=5, warmup_steps=500,
    per_device_train_batch_size=4, per_device_eval_batch_size=4,
    weight_decay=0.01, logging_steps=10,
    evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16
)

In [ ]:
trainer = Trainer(model=model_bart, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=df_train,
                  eval_dataset=df_test)

In [ ]:
trainer.train()

In [ ]:
!pip install wandb
import wandb
!wandb login

In [ ]:
class scientificPaperSummary(Dataset):

  def __init__(self, DataFrame, tokenizer , text_max_token_len, summary_max_token_len):

    self.tokenizer = tokenizer
    self.data = DataFrame
    self.article = self.data.article
    self.abstract = self.data.abstract
    self.text_max_token_len = text_max_token_len
    self.summary_max_token_len = summary_max_token_len

  def __len__(self):
    return len(self.article)

  def __getitem__ (self, index):
    abstract = str(self.abstract[index])
    abstract = ' '.join(abstract.split())

    article = str(self.article[index])
    article = ' '.join(article.split())


    summary_encoding = self.tokenizer.batch_encode_plus([abstract], max_length= self.text_max_token_len, truncation = True,return_tensors='pt')
    text_encoding = self.tokenizer.batch_encode_plus([article], max_length= self.summary_max_token_len,truncation = True,return_tensors='pt')

    summary_ids = summary_encoding["input_ids"].squeeze()
    summary_mask = summary_encoding["attention_mask"].squeeze()
    article_ids = text_encoding["input_ids"].squeeze()
    article_mask = text_encoding["attention_mask"].squeeze()



    return {
        'summary_ids' : summary_ids.to(dtype = torch.long),
        'summary_mask' : summary_mask.to(dtype = torch.long),
        'article_ids' : article_ids.to(dtype = torch.long),
        'article_mask' : article_mask.to(dtype = torch.long)

    }


In [ ]:
# Creating the training function. This will be called in the main function. It is run depending on the epoch value.
# The model is put into train mode and then we wnumerate over the training loader and passed to the defined network
import torch.nn.functional as F

def train(epoch, tokenizer, model, device, loader, optimizer):
    model.train()
    for _,data in enumerate(loader, 0):
        y = data['target_ids'].to(device, dtype = torch.long)
        y_ids = y[:, :-1].contiguous()
        lm_labels = y[:, 1:].clone().detach()
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data['source_ids'].to(device, dtype = torch.long)
        mask = data['source_mask'].to(device, dtype = torch.long)

        outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, lm_labels=lm_labels)
        loss = outputs[0]


        if _%10 == 0:
            wandb.log({"Training Loss": loss.item()})

        if _%500==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


        # xm.optimizer_step(optimizer)
        # xm.mark_step()

In [ ]:
def validate(epoch, tokenizer, model, device, loader):
  model.eval()
  predictions = []
  actuals = []
  with torch.no_grad():
    for _, data in enumerate(loader, 0):
      y = data['summary_ids'].to(device, dtype = torch.long)
      ids = data['article_ids'].to(device, dtype = torch.long)
      mask = data['article_mask'].to(device, dtype = torch.long)

      generated_ids = model.generate(
          input_ids = ids,
          attention_mask = mask,
          max_length=150,
          num_beams=2,
          repetition_penalty=2.5,
          length_penalty=1.0,
          early_stopping=True)
      preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
      target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
      if _%100==0:
        print(f'Completed {_}')
      predictions.extend(preds)
      actuals.extend(target)
  return predictions, actuals

In [ ]:
MODEL_NAME = "t5-small"


In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'



In [ ]:
import tensorflow as tf

In [ ]:
def main():

  wandb.init(project="transformers_tutorials_summarization")

  # WandB – Config is a variable that holds and saves hyperparameters and inputs
  # Defining some key variables that will be used later on in the training
  config = wandb.config          # Initialize config
  config.TRAIN_BATCH_SIZE = 2    # input batch size for training (default: 64)
  config.VALID_BATCH_SIZE = 2    # input batch size for testing (default: 1000)
  config.TRAIN_EPOCHS = 2        # number of epochs to train (default: 10)
  config.VAL_EPOCHS = 1
  config.LEARNING_RATE = 1e-4    # learning rate (default: 0.01)
  config.SEED = 42               # random seed (default: 42)
  config.MAX_LEN = 512
  config.SUMMARY_LEN = 150

    # Set random seeds and deterministic pytorch for reproducibility
  torch.manual_seed(config.SEED) # pytorch random seed
  np.random.seed(config.SEED) # numpy random seed
  torch.backends.cudnn.deterministic = True

  tokenizer = T5Tokenizer.from_pretrained("t5-base")

  # Creating the Training and Validation dataset for further creation of Dataloader
  training_set = scientificPaperSummary(df_train, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)
  val_set = scientificPaperSummary(df_test, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)

  # Defining the parameters for creation of dataloaders
  # Defining the parameters for creation of dataloaders
  train_params = {
      'batch_size': config.TRAIN_BATCH_SIZE,
      'shuffle': True,
      'num_workers': 0}

  val_params = {
      'batch_size': config.VALID_BATCH_SIZE,
      'shuffle': False,
      'num_workers': 0}



  # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
  training_loader = DataLoader(df_train, **train_params)
  val_loader = DataLoader(df_test, **val_params)

  model = T5ForConditionalGeneration.from_pretrained('t5-base')

  model = model.to(device)
      # Defining the optimizer that will be used to tune the weights of the network in the training session.
  optimizer = torch.optim.Adam(params =  model.parameters(), lr=config.LEARNING_RATE)

    # Log metrics with wandb
  wandb.watch(model, log="all")
    # Training loop
  print('Initiating Fine-Tuning for the model on our dataset')

  for epoch in range(config.TRAIN_EPOCHS):
    train(epoch, tokenizer, model, device, training_loader, optimizer)


    # Validation loop and saving the resulting file with predictions and acutals in a dataframe.
    # Saving the dataframe as predictions.csv
  print('Now generating summaries on our fine tuned model for the validation dataset and saving it in a dataframe')
  for epoch in range(config.VAL_EPOCHS):
    predictions, actuals = validate(epoch, tokenizer, device, model, val_loader)
    final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})
    final_df.to_csv('./models/predictions.csv')
    print('Output Files generated for review')

if __name__ == '__main__':
  main()

In [ ]:
print(logits.shape, lm_labels.shape)

In [ ]:
class sciSummaryDataModule(pl.LightningDataModule):
  def __init__ (
      self,
      df_train = pd.DataFrame,
      df_test = pd.DataFrame,
      tokenizer = T5Tokenizer,
      batch_size = 32,
      text_max_token_len = 512,
      summary_max_token_len = 128
  ):



      super().__init__()

      self.df_train = df_train
      self.df_test = df_test

      self.batch_size = batch_size
      self.tokenizer = tokenizer
      self.text_max_token_len = text_max_token_len
      self.summary_max_token_len = summary_max_token_len

  def setup(self, stage = None):
    self.train_dataset = scientificPaperSummary(
        self.df_train,
        self.tokenizer,
        self.text_max_token_len,
        self.summary_max_token_len
    )

    self.test_dataset = scientificPaperSummary(
        self.df_test,
        self.tokenizer,
        self.text_max_token_len,
        self.summary_max_token_len
    )

  def train_dataloader(self):
    return Dataset(
        self.train_dataset,
        batch_size = self.batch_size,
        shuffle = True,
        num_worker =2
    )

  def test_dataloader(self):
    return Dataset(
        self.test_dataset,
        batch_size = self.batch_size,
        shuffle = False,
        num_worker =2
    )

In [ ]:
text_token_counts, summary_token_counts= [], []

for _, row in df_train.iterrows():
  text_token_count = len(tokenizer.encode(row["article"]))
  text_token_counts.append(text_token_count)

  summary_token_count = len(tokenizer.encode(row["abstract"]))
  summary_token_counts.append(summary_token_count)

In [ ]:
summary_token_counts

In [ ]:
# Plot summary_token_counts in one figure
plt.figure(figsize=(8, 6))
plt.plot(summary_token_counts, marker='o', color='b', label='Summary Token Counts')
plt.xlabel('Number of papers')
plt.ylabel('Counts')
plt.title('Summary Token Counts')
plt.legend()
plt.grid(True)
plt.show()

# Plot text_token_counts in another figure
plt.figure(figsize=(8, 6))
plt.plot(text_token_counts, marker='o', color='g', label='Text Token Counts')
plt.xlabel('Number of papers')
plt.ylabel('Counts')
plt.title('Text Token Counts')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
class SciPaperSummaryModel(pl.LightningModule):

    def __init__(self):
        super().__init__()
        self.model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME, return_dict=True)

    def forward(self, input_ids, attention_mask, decoder_attention_mask, labels=None):
        output = self.model(input_ids, attention_mask=attention_mask, labels=labels, decoder_attention_mask=decoder_attention_mask)
        return output.loss, output.logits

    def training_step(self, batch, batch_idx):
        input_ids = batch["text_input_ids"]
        attention_mask = batch["text_attention_mask"]
        labels = batch["labels"]
        labels_attention_mask = batch["labels_attention_mask"]

        loss, outputs = self(input_ids=input_ids, attention_mask=attention_mask, decoder_attention_mask=labels_attention_mask, labels=labels)
        self.log("train_loss", loss, prog_bar=True, logger=True)
        return loss

    def test_step(self, batch, batch_idx):
        input_ids = batch["text_input_ids"]
        attention_mask = batch["text_attention_mask"]
        labels = batch["labels"]
        labels_attention_mask = batch["labels_attention_mask"]

        loss, outputs = self(input_ids=input_ids, attention_mask=attention_mask, decoder_attention_mask=labels_attention_mask, labels=labels)
        self.log("test_loss", loss, prog_bar=True, logger=True)
        return loss

    def configure_optimizers(self):
        return AdamW(self.parameters(), lr=0.0001)


In [ ]:
%matplotlib inline

In [ ]:
n_epoch = 10
batch_size = 32

data_module = sciSummaryDataModule(df_train, df_test, tokenizer, batch_size = batch_size)